## Remap events.
This notebook demonstrates how to use HED tools to remap event files. The idea of
remapping is that each unique combination of values in a set of **key** columns is
mapped into a specified combination of values in a set of **target** columns.

The example used in this notebook is a Sternberg memory dataset. A short dataset is:
contained in the `data/sternberg` directory.  In the example, each value in the
`type` column is mapped to a combination of values in the `event_type`, `task_role`,
and `letter` columns. Since the event files don't contain these columns, the mapping
inserts these columns as part of the mapping.


### Step 1: Read in a mapping that has already been created

If you don't have a mapping for your data, you should use the
`hed_evaluate_events.ipynb` notebook first. The following reads in
the mapping designed to insert `event_type`, `task_role`,
and `letter` columns corresponding to values in the `type` column.


In [1]:
import os
from hed.tools import KeyMap
key_cols = ['type']
target_cols = ['event_type', 'task_role', 'letter']
name = 'sternberg_remap'
root_path = "../data/sternberg"
remap_file = os.path.join(root_path, "sternberg_map.tsv")
key_map = KeyMap(key_cols, target_cols, name)
duplicates = key_map.update(remap_file)
if duplicates:
    print(f"Warning: this mapping has {len(duplicates)} duplicate key entries")

### Step 2: Remap event files using the mapping

Remapping consists of replacing/adding columns (in this case adding columns).

In [2]:
from hed.tools import generate_filename, get_file_list
event_file_list = get_file_list(root_path, types=[".tsv"], suffix="_events")

for file in event_file_list:
    print(f"Remapping {file}...")
    df_new, missing = key_map.remap(file)
    if missing:
        print(f"The keys {str(missing)} were not in the map for {file}")
    file_name = generate_filename(os.path.basename(file), prefix="remapped", extension='.tsv')
    print(f"...Writing remapped file to {file_name}")
    df_new.to_csv(file_name, sep='\t', index=False)

Remapping ../data/sternberg\sternberg_no_quotes_events.tsv...
...Writing remapped file to remapped_sternberg_no_quotes_events.tsv
Remapping ../data/sternberg\sternberg_test_events.tsv...
...Writing remapped file to remapped_sternberg_test_events.tsv
Remapping ../data/sternberg\sternberg_with_quotes_events.tsv...
...Writing remapped file to remapped_sternberg_with_quotes_events.tsv
